In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
from tqdm import tqdm_notebook as tqdm
import pickle
from pathlib import Path
from nuscenes import NuScenes
from scipy.spatial.transform import Rotation as R 
from math import cos, sin, pi
from lyft_dataset_sdk.lyftdataset import *
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
from lyft_dataset_sdk.eval.detection.mAP_evaluation import Box3D, recall_precision

In [3]:
phase = 'train'
data = 'v1.0-trainval' if phase=='train' else 'v1.0-test'
lyft = LyftDataset(data_path=f'../../../data/lyft/{phase}/', json_path=f'../../../data/lyft/{phase}/{data}/', verbose=0) 
# nusc = NuScenes(dataroot=f'../../data/lyft/{phase}/', version=data, verbose=0) 

### prepare ground truth json files

Ground truth json files for train/val set are used for evaluation using  lyft's official mAP metric

In [8]:
# info_path = '../../../data/lyft/train/infos_val.pkl'
info_path = '../../../data/lyft/train/infos_train.pkl'
with open(info_path, 'rb') as f:
    info = pickle.load(f)['infos']

In [9]:
gt_data = [] 
for inf in tqdm(info):
    sample_token = inf['token']
    sample = lyft.get('sample', sample_token)
    for ann_token in sample['anns']:
        ann_record = lyft.get('sample_annotation', ann_token)
        data = {
            'sample_token': sample_token,
            'translation': ann_record['translation'],
            'size': ann_record['size'],
            'rotation': ann_record['rotation'],
            'name': ann_record['category_name']
        }
        gt_data.append(data)

100%|██████████| 18522/18522 [00:01<00:00, 10945.87it/s]


In [10]:
len(gt_data), gt_data[0]

(524283,
 {'sample_token': 'b98a05255ba2632e957884758cb31f0e6fcc8d3cd6ee76b6d0ba55b72f08fc54',
  'translation': [1134.216191599088, 1650.4749269743418, -23.38887732990988],
  'size': [1.889, 4.852, 1.829],
  'rotation': [0.27834486968982886, 0, 0, 0.9604811989400689],
  'name': 'car'})

In [11]:
# gt_data_path = '../../../data/lyft/train/gt_data_val.json'
gt_data_path = '../../../data/lyft/train/gt_data_train.json'
with open(gt_data_path, 'w') as f:
    json.dump(gt_data, f, indent=2)